## Basic data import 

Basic data import from Garmin fit files. All main fields for a bicicle race, including also some of the marks for laps. Regular laps and special lap treatment with stops here. The fields are for bicicle race but could also apply to running with some exceptions and we would need to add those more often used to running like pace. If you need access to other data, there is more on strucutre and contents of fit files in another notebook...

Data can come from multiple users or multiple events. Each user and event in a separate file. In this one event with multiple participants. File name corresponds to some form of ID of a user...

We start by doing basic imports. In particular we will use fitparse library to read contents of fit files...

In [ ]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fitparse import FitFile
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from collections import OrderedDict

we assume all files are in the same directory. we will load main information that's from along that race separatly from information about laps that. could this be influences by autolaps...

In [ ]:
participant_files = [f for f in listdir('../needed-2018/T2') if isfile(join('../needed-2018/T2', f))]
for i in range(len(participant_files)):
    participant_files[i]
all_dfs = []
all_stops = []
all_dStops = []
all_rDfs = []

Reading data for one user at a time with lap and stop information. 



display start - end time 
messages of type event, event_type: start and stop_all


list of laps (start - end time)

main fields that carry time information are start_time, timestamp, total_elapsed_time

total_timer_time is total_elapsed_time - auto pause time (if in use), ref: https://forums.garmin.com/forum/into-sports/cycling/edge-810-aa/45987-...

Total elapsed time on the last lap is not the same as timestamp-start_time, it also includes the time from when activity was stop to the moment that the activity is saved... 

speed corr function ensures that we get a float even if there is not number. otherwise, single such execpetion could detrail the whole input procedure. make sure that convertin lack of value to 0 makes sense in your case...

we define two additional functions, one for reading information from a record message that contains all the tracking information such as timestamp, speed, distance etc. the second function to read information from lap message that is recorded i.a. every time user presses lap button...

why do we have type information, could we drop it and just separate data?

In [ ]:
def speed_corr(speed):
    return speed if isinstance(speed, float) else 0

def power_corr(power):
    return power if isinstance(power, int) else 0

def read_record(rec):
    global curr_stop
    #global curr_active
    if curr_stop == 1 and speed_corr(rec.get_value('speed')) > 7:
        curr_stop = 0
    #if ((speed_corr(rec.get_value('speed')) > 0) & (pd.isna(rec.get_value('power'))) or ((speed_corr(rec.get_value('speed')) > 0) & (power_corr(rec.get_value('power'))> 0))):
        #curr_active = 1
    return (rec.get_value('timestamp'), 1, curr_lap, curr_stop,
            rec.get_value('speed'), rec.get_value('enhanced_speed'),
            rec.get_value('distance'), rec.get_value('heart_rate'), 
            rec.get_value('accumulated_power'), rec.get_value('power'), 
            rec.get_value('position_lat'), rec.get_value('position_long'), 
            rec.get_value('altitude'), rec.get_value('enhanced_altitude'), 
            rec.get_value('cadence'), rec.get_value('factional_cadence'), 
            rec.get_value('left_right_balnce'), rec.get_value('left_torque_effectiveness'), 
            rec.get_value('right_torque_effectiveness'), rec.get_value('left_pedal_smoothness'), 
            rec.get_value('right_pedal_smoothness'), rec.get_value('temperature'))


def read_lap(rec):
    global curr_lap
    global curr_stop
    curr_lap += 1
    curr_stop = 1
    return (rec.get_value('timestamp'), 2, curr_lap, curr_stop, np.NaN, np.NaN, rec.get_value('total_distance'), np.NaN, np.NaN, 
            np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
            np.NaN, np.NaN, np.NaN, np.NaN, np.NaN)

fitgen_cols = ['timestamp', 'type', 'lap', 'stop', 'speed', 'enhanced_speed', 'distance', 'heart_rate', 
               'accumulated_power', 'power', 'position_lat', 'position_long', 
               'altitude', 'enhanced_altitude', 'cadence', 'factional_cadence', 
               'left_right_balnce', 'left_torque_effectiveness', 'right_torque_effectiveness', 
               'left_pedal_smoothness', 'right_pedal_smoothness', 'temperature']

#initialize begin end as empty timestamps
#start = pd.Timestamp(2020, 1, 1, 12)
#end = pd.Timestamp(2016, 1, 1, 12)

for participant_file in participant_files:
    fitfile = FitFile('../needed-2018/T2/'+participant_file)
    print(participant_file, 'start')
    
    curr_lap = 0
    curr_stop = 0
    #curr_active = 0
    
    fitgen = (read_record(rec) if rec.name=='record' else read_lap(rec) for 
    rec in fitfile.get_messages(['record', 'lap']))

    allraw = list(rec for rec in fitfile.get_messages(['record', 'lap']))

    print(participant_file, 'read')
    
    part_df = pd.DataFrame(fitgen, columns=fitgen_cols)
    all_dfs.append(part_df[part_df['type']==1].set_index('timestamp'))
    all_stops.append(part_df[part_df['type']==2].set_index('timestamp'))
    
    #relativ informasjon. Blir litt feil.
    #all_rDfs.append(part_df[part_df['active']==1].set_index('timestamp'))

    #if all_dfs[-1].index[0]<start: start = all_dfs[-1].index[0]
    #if all_dfs[-1].index[-1]>end: end = all_dfs[-1].index[-1]
    
    print(participant_file, 'converted')


#f = open('all_rDfs.txt', 'w')
#for element in all_rDfs:
 #   f.write(element.to_string())
  #  f.write('\n')
#f.close()
#new_index = pd.date_range(start=start, end=end, freq='s')
#for i in range(len(all_dfs)): #regular for loop without counter doesn't work
    #all_dfs[i] = all_dfs[i].reindex(new_index)

Creates all_dStops.

In [ ]:
#indexarray also used in last stop plot.
indexArray = []
tempIndexes = []
tempStops=[]

#kanskje endre til -29 +29.
def lookup_dfs(df, i, z):
    stopStart = df[i].index.get_loc(indexArray[i][z])
    tempDF = df[i].iloc[stopStart+1:]
    stopEnd = df[i].index.get_loc(tempDF[(tempDF['stop'].diff() != 0) & (tempDF['lap'].diff() == 0) & (tempDF['lap'] == z+1)].index[0])
    return df[i].iloc[stopStart-30:stopEnd+30]
    

#bruker -1 , men MÅ endre. Gjør pga stuck, siste måling virker spesiell..
for i in range(len(all_stops)):
    for x in range(len(all_stops[i])-1):
        tempIndexes.append(all_stops[i].index[x])
    indexArray.append(tempIndexes)
    tempIndexes = []
    for z in range(len(indexArray[i])):
        tempStops.append(lookup_dfs(all_dfs, i, z))
    userStop = pd.concat(tempStops)
    all_dStops.append(userStop)
    tempStops = []


In [ ]:
f = open('all_dStops.txt', 'w')
for element in all_dStops:
    f.write(element.to_string())
    f.write('\n')
f.close()

In [ ]:
#allraw

In [ ]:
#add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

In [ ]:
#stops_df = pd.concat(all_stops, axis=1, keys=[a[:-4] for a in participant_files])

In [ ]:
#add_df['50338'][['stop','lap']].dropna().plot()

In [ ]:
#add_df.swaplevel(axis=1)['stop'].sum().mean()
#add_df.swaplevel(axis=1)['stop'].sum().median()
#add_df.swaplevel(axis=1)['stop'].sum().min()
#add_df.swaplevel(axis=1)['stop'].sum().max()

In [ ]:
#stops_df['50338'][['stop','lap']].dropna().plot()

In [ ]:
#stops_df['50338'][['stop','lap']].dropna()

In [ ]:
#add_df.to_pickle('../all_df.pickle')
#stops_df.to_pickle('../stops_df.pickle')

In [ ]:
#Turns off useless spines, ticks and labels for additional axis.
#Turns on the spine we want to see, and position it.
def ax_modification(ax,pos,val):
    for value in ax.spines.values():
        value.set_visible(False)
    ax.tick_params(axis='x', bottom=False, labelbottom=False)
    ax.spines[pos].set_visible(True)
    ax.spines[pos].set_position(('axes',val))
    ax.yaxis.set_label_position(pos)
    ax.yaxis.set_ticks_position(pos)
    if val >= 1:
        val += 0.15
    else:
        val -= 0.15
    return val

def ax_style(ax,title,color):
    ax.set_ylabel(title)
    ax.yaxis.label.set_color(color)
    ax.tick_params(axis='y', colors=color)

In [ ]:
#Axis for 10 minutes start for every participant.
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.15
    #first ax and figure stretch.
    ID = participant_file[:-4]
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)
    #number of additional y axis:
    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()

    #position spines and ticks for axis.
    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)
    rightPosition = ax_modification(ax5,'right', rightPosition)


    #Finds start where not only NaN:
    startDate = add_df[ID].index[~add_df[ID].isnull().all(1)]
    startRow = add_df[ID].index.get_loc(startDate[0])    
    endRow = startRow + 600

    add_df[ID]['accumulated_power'].iloc[startRow:endRow].plot(ax=ax, color='red',legend=False, title= ID + " 10 minutes start")
    add_df[ID]['speed'].iloc[startRow:endRow].plot(ax=ax2, color='blue',legend=False)
    add_df[ID]['power'].iloc[startRow:endRow].rolling(window=30).mean().plot(ax=ax3, color='green',legend=False)
    add_df[ID]['altitude'].iloc[startRow:endRow].plot(ax=ax4, color='orange',legend=False)
    add_df[ID]['distance'].diff().iloc[startRow:endRow].plot(ax=ax5, color='lightcoral',legend=False)

    ax_style(ax,'Accumulated Power','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Altitude','orange')
    ax_style(ax5,'Distance change','lightcoral')

In [ ]:
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.15

    ID = participant_file[:-4]
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)

    add_df[ID][['heart_rate']].plot(ax=ax, color='red',legend=False, title= ID + " Relativ Start")
    add_df[ID][['speed']].plot(ax=ax2, color='blue',legend=False)
    add_df[ID][['power']].rolling(window=30).mean().plot(ax=ax3, color='green',legend=False)
    add_df[ID][['distance']].plot(ax=ax4, color='orange',legend=False)

    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Distance','orange')

    #ax4.legend([ax.get_lines()[0], ax2.get_lines()[0], ax3.get_lines()[0], ax4.get_lines()[0]], ['Heart rate','Speed','MA Power','Distance'], loc='lower left', bbox_to_anchor=(1.3, 0.5))


In [ ]:
all_dStops_df = pd.concat(all_dStops, axis=1, keys=[a[:-4] for a in participant_files])
all_dStops_df.index = map(lambda x: str(x)[-8:], all_dStops_df.index)
for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.15
    ID = participant_file[:-4]

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    rightPosition = ax_modification(ax4, 'right', rightPosition)
    leftPosition = ax_modification(ax5,'left', leftPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    stopValues = all_dStops_df[ID][~all_dStops_df[ID].isnull().all(1)]

    stop_values[['heart_rate']].plot(ax=ax, color='red',legend=False, title= ID + " All stops")
    stop_values[['speed']].plot(ax=ax2, color='blue',legend=False)
    stop_values[['power']].plot(ax=ax3, color='green',legend=False)
    stop_values[['distance']].plot(ax=ax4, color='orange',legend=False)
    stop_values[['altitude']].plot(ax=ax5, color='lightcoral',legend=False)
    stop_values[['accumulated_power']].plot(ax=ax6, color='purple',legend=False)

    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'Power','green')
    ax_style(ax4,'Distance change','orange')
    ax_style(ax5,'Altitude','lightcoral')
    ax_style(ax6,'Accumulated Power','purple')

In [ ]:
all_dStops_df = pd.concat(all_dStops, axis=1, keys=[a[:-4] for a in participant_files])

for participant_file in participant_files:
    rightPosition = 1.00
    leftPosition = -0.15
    ID = participant_file[:-4]

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    rightPosition = ax_modification(ax4, 'right', rightPosition)
    leftPosition = ax_modification(ax5,'left', leftPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    #finding start and endDate for vertical line. Rows for iloc.
    descendingDF = all_dStops_df[ID].sort_index(ascending=False)
    startDate = all_dStops_df[ID]['lap'].idxmax()
    endDate = descendingDF['lap'].idxmax() - pd.Timedelta(seconds=30)

    startRow = all_dStops_df[ID].index.get_loc(startDate)-31
    endRow = all_dStops_df[ID].index.get_loc(endDate)+30


    all_dStops_df[ID][['heart_rate']].iloc[startRow:endRow].plot(ax=ax, color='red',legend=False, title= ID + " Last stop")
    all_dStops_df[ID][['speed']].iloc[startRow:endRow].plot(ax=ax2, color='blue',legend=False)
    all_dStops_df[ID][['power']].iloc[startRow:endRow].plot(ax=ax3, color='green',legend=False)
    all_dStops_df[ID][['distance']].iloc[startRow:endRow].diff().plot(ax=ax4, color='orange',legend=False)
    all_dStops_df[ID][['altitude']].iloc[startRow:endRow].plot(ax=ax5, color='lightcoral',legend=False)
    all_dStops_df[ID][['accumulated_power']].iloc[startRow:endRow].plot(ax=ax6, color='purple',legend=False)

    ax_style(ax,'Heart rate','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'MA Power','green')
    ax_style(ax4,'Distance change','orange')
    ax_style(ax5,'Altitude','lightcoral')
    ax_style(ax6,'Accumulated Power','purple')

    ax.axvline(startDate, color='black', linestyle='--', lw=2)
    ax.axvline(endDate, color='black', linestyle='--', lw=2)